In [63]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime

dataDir = "/code/archive/garbage_classification/Garbage_classification"

imgHeight = 224
imgWidth = 224
batchSize = 32

trainDs = tf.keras.utils.image_dataset_from_directory(
  dataDir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(imgHeight, imgWidth),
  batch_size=batchSize)

valDs = tf.keras.utils.image_dataset_from_directory(
  dataDir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(imgHeight, imgWidth),
  batch_size=batchSize)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"

feature_extractor_model = mobilenet_v2

IMAGE_SHAPE = (imgHeight, imgWidth)

# Apply normalization to images
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = trainDs.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = valDs.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

AUTOTUNE = tf.data.AUTOTUNE
trainDs = trainDs.cache().prefetch(buffer_size=AUTOTUNE)
valDs = valDs.cache().prefetch(buffer_size=AUTOTUNE)

resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.Resizing(imgHeight,imgWidth),
])

feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(imgHeight, imgWidth, 3),
    trainable=False)

feature_extractor_layer.trainable = False


model = tf.keras.Sequential([
  resize_and_rescale,
  feature_extractor_layer,
  tf.keras.layers.Dense(5)
])

model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])
# model.summary()




Found 2390 files belonging to 5 classes.
Using 1912 files for training.
Found 2390 files belonging to 5 classes.
Using 478 files for validation.


In [65]:
# Fit model

history = model.fit(trainDs,
epochs=5,
validation_data=valDs)

# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

Epoch 1/5
60/60 [==============================] - 22s 365ms/step - loss: 0.7003 - acc: 0.7641 - val_loss: 0.7849 - val_acc: 0.7008
Epoch 2/5
60/60 [==============================] - 22s 367ms/step - loss: 0.6660 - acc: 0.7793 - val_loss: 0.7720 - val_acc: 0.7071
Epoch 3/5
60/60 [==============================] - 22s 366ms/step - loss: 0.6362 - acc: 0.7887 - val_loss: 0.7617 - val_acc: 0.7071
Epoch 4/5
60/60 [==============================] - 22s 363ms/step - loss: 0.6099 - acc: 0.7981 - val_loss: 0.7535 - val_acc: 0.7155
Epoch 5/5
60/60 [==============================] - 22s 364ms/step - loss: 0.5863 - acc: 0.8049 - val_loss: 0.7467 - val_acc: 0.7155


In [1]:
from PIL import Image
model.save('./saved_mode/garbage_class.h5')
# img = Image.open("/code/test_images/plastic_bottle_1.jpg").convert('L').resize((imgWidth,imgHeight), Image.ANTIALIAS)
# img = np.array(img)
# print(img.shape())

img = tf.io.read_file("/code/test_images/cardboard1.jpg")
tensor = tf.io.decode_image(img, channels=3, dtype=tf.dtypes.float32)
tensor = tf.image.resize(tensor, [imgHeight, imgWidth])
input_tensor = tf.expand_dims(tensor, axis=0)

print(model.predict(input_tensor))
print(max(model.predict(input_tensor)[0]))

NameError: name 'model' is not defined